In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plot
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
from matplotlib.dates import (AutoDateLocator, YearLocator, MonthLocator,
                              DayLocator, WeekdayLocator, HourLocator,
                              MinuteLocator, SecondLocator, MicrosecondLocator,
                              RRuleLocator, rrulewrapper, MONTHLY,
                              MO, TU, WE, TH, FR, SA, SU, DateFormatter,
                              AutoDateFormatter, ConciseDateFormatter)
import datetime
from os import listdir, walk
from os.path import isfile, join
import xlsxwriter
from bokeh.plotting import figure, show

In [3]:

testplan_path ='//DLA-04/Shared/KAILUA PROJECTS/2023/23-001 Hickam AFB Acoustical Testing/Documents/Python_devstuff/INCOMING - PAFWC STC Testing Plan List_SRBR_Aprilrev1.xlsx'
# write an input here
# testplanfile = openpyxl.load_workbook(testplan_path)
testplanfile = pd.read_excel(testplan_path)
# copy all entries from  testplan and put them in a list 
# testlist = testplanfile.active


In [4]:

# test_num = testplanfile['Test Label']
# source_slm_num = testplanfile['Source']
# receive_slm_num = testplanfile['Recieve ']
# bkgnd_slm_num = testplanfile['BNL']
# rt_slm_num = testplanfile['RT']
# ASTC_enable = testplanfile['ASTC?']
# NIC_enable = testplanfile['NIC?']
# path for SLM raw data - use forward slash for windows paths
# change these for each project files to add 
rawDtestpath = '//DLA-04/Shared/KAILUA PROJECTS/2023/23-001 Hickam AFB Acoustical Testing/Test Data/Raw Data/SLM D - 3784/'
rawEtestpath = '//DLA-04/Shared/KAILUA PROJECTS/2023/23-001 Hickam AFB Acoustical Testing/Test Data/Raw Data/SLM E - 4328/'
D_datafiles = [f for f in listdir(rawDtestpath) if isfile(join(rawDtestpath,f))]
E_datafiles = [f for f in listdir(rawEtestpath) if isfile(join(rawEtestpath,f))]
# depreciated - plan is to calc and report numbers purely from the data 
# rawReportpath = '//DLA-04/Shared/KAILUA PROJECTS/2023/23-001 Hickam AFB Acoustical Testing/Documents/Python_devstuff/Report_4-7-23/'
# reports =  [f for f in listdir(rawReportpath) if isfile(join(rawReportpath,f))]
# #only read excel files, filter for .xlsx
# reports = [k for k in reports if '.xlsx' in k]
# print(reports)

In [5]:
# single test find 
find_test = '0.1.1'
mask = testplanfile.applymap(lambda x: find_test in x if isinstance(x,str) else False).to_numpy()
indices = np.argwhere(mask) 
# print(indices)
index = indices[0,0]
# print(index)
print(testplanfile.iloc[index])
foundtest = testplanfile.iloc[index]

Area                NaN
Area Label          NaN
Test Label        0.1.1
Source Room         NaN
Receiving Room      NaN
Partition Type      NaN
Design STC          NaN
Test Date           NaN
Unnamed: 8          NaN
Test Report by      NaN
Source             D600
Recieve            E042
BNL                E041
RT                 E072
ASTC?               0.0
NIC?                1.0
room volume         NaN
Name: 0, dtype: object


In [6]:
srs_slm_file = list()
receive_slm_file = list()
bkgrnd_slm_file = list()
rt_slm_file = list()
# found test iteration, find Source, Recieve, BNL, RT files 
currsrs = foundtest['Source']
if currsrs[0] == 'D':
        datafile_num = currsrs[1:]
        ##D meter, pull D_datafiles
            ## (D_datafiles) for filename
        datafile_num = datafile_num+'.xlsx'
        srs_slm_found = [x for x in D_datafiles if datafile_num in x]
        # print(srs_slm_found)
        srs_slm_file.append(rawDtestpath+srs_slm_found[0])
elif currsrs[0] == 'E':
        datafile_num = currsrs[1:]
        ##E meter, pull E_datafiles
        datafile_num = datafile_num+'.xlsx'
        srs_slm_found = [x for x in E_datafiles if datafile_num in x]
        # print(rec_slm_found)
        srs_slm_file.append(rawEtestpath+srs_slm_found[0])
# file to read is srs
currrec = foundtest['Recieve '] 
if currrec[0] == 'D':
        datafile_num = currrec[1:]
        ##D meter, pull D_datafiles
            ## (D_datafiles) for filename
        datafile_num = datafile_num+'.xlsx'
        rec_slm_found = [x for x in D_datafiles if datafile_num in x]
        # print(rec_slm_found)
        receive_slm_file.append(rawDtestpath+rec_slm_found[0])
elif currrec[0] == 'E':
        datafile_num = currrec[1:]
        ##E meter, pull E_datafiles
        datafile_num = datafile_num+'.xlsx'
        rec_slm_found = [x for x in E_datafiles if datafile_num in x]
        # print(rec_slm_found)
        receive_slm_file.append(rawEtestpath+rec_slm_found[0])

bkgnd_list = foundtest['BNL']  
if bkgnd_list[0] == 'D':
        datafile_num = bkgnd_list[1:]
        ##D meter, pull D_datafiles
        ## (D_datafiles) for filename
        datafile_num = datafile_num+'.xlsx'
        bkgrnd_slm_found = [x for x in D_datafiles if datafile_num in x]
        bkgrnd_slm_file.append(rawDtestpath+bkgrnd_slm_found[0])
elif bkgnd_list[0] == 'E':
        datafile_num = bkgnd_list[1:]
        ##E meter, pull E_datafiles
        datafile_num = datafile_num+'.xlsx'
        bkgrnd_slm_found = [x for x in E_datafiles if datafile_num in x]
        bkgrnd_slm_file.append(rawEtestpath+bkgrnd_slm_found[0])
rt_list = foundtest['RT']
if rt_list[0] == 'D':
        datafile_num = rt_list[1:]
        # print(datafile_num)
        ##D meter, pull D_datafiles
            ## (D_datafiles) for filename
        datafile_num = datafile_num+'.xlsx'
        rt_slm_found = [x for x in D_datafiles if datafile_num in x]
        # print(rec_slm_found)
        rt_slm_file.append(rawDtestpath+rt_slm_found[0])
elif rt_list[0] == 'E':
        datafile_num = rt_list[1:]
        # print(datafile_num)
        ##E meter, pull E_datafiles
        datafile_num = datafile_num+'.xlsx'
        rt_slm_found = [x for x in E_datafiles if datafile_num in x]
        # print(rec_slm_found)
        rt_slm_file.append(rawEtestpath+rt_slm_found[0])


In [40]:
OBAdatasheet = 'OBA'
RTsummarysheet = 'Summary'
srs_OBAdata = pd.read_excel(srs_slm_file[0],OBAdatasheet)
recive_OBAdata = pd.read_excel(receive_slm_file[0],OBAdatasheet)
bkgrd_OBAdata = pd.read_excel(bkgrnd_slm_file[0],OBAdatasheet)
rt = pd.read_excel(rt_slm_file[0],RTsummarysheet)

In [41]:
#pulling raw OBA data from files 
srs_overalloct = srs_OBAdata.iloc[7]
srs_overalloct = srs_overalloct[14:30]
rec_overalloct = recive_OBAdata.iloc[7]
rec_overalloct = rec_overalloct[14:30]
bkgrnd_overalloct = bkgrd_OBAdata.iloc[7]
bkgrnd_overalloct = bkgrnd_overalloct[14:30]
rt_thirty = rt['Unnamed: 10']
rt_thirty = rt_thirty[25:41]
rt_thirty = rt_thirty/1000

In [42]:
#calcs - sabines, ATL, Deficiencies 

#room volumes in cubic meters
recieve_roomvol = 15.3378
source_roomvol = 193.141
parition_area = 4.64
ASTC_vollimit = 883
if recieve_roomvol > ASTC_vollimit:
    print('Using NIC calc, room volume too large')

constant = np.int32(20.047*np.sqrt(273.15+20))
#constant contour 
# STC_contour = [28,31,34,37,40,43,44,45,46,47,48,48,48,48,48,48]
# rt_thirty = np.array(rt_thirty, dtype=int)
# print(rt_thirty)
intermed = 30/rt_thirty
thisval = np.int32(recieve_roomvol*intermed)
sabines =thisval/constant
sabines = np.round(sabines*(0.921))
# reciever correction 
# print('sabines: ', sabines)
recieve_corr = list()
recieve_vsBkgrnd = rec_overalloct - bkgrnd_overalloct
print('rec vs background:',recieve_vsBkgrnd)
for i in range(len(recieve_vsBkgrnd)):
    val = recieve_vsBkgrnd[i]
    print('val:', recieve_vsBkgrnd[i])
    print('count: ',i)
    if val < 5:
        recieve_corr.append(rec_overalloct.iloc[i]-2)
    elif val < 10:
        recieve_corr.append(np.log10(10**(rec_overalloct.iloc[i]/10)-10**(bkgrnd_overalloct.iloc[i]/10)))
    else:
        recieve_corr.append(rec_overalloct.iloc[i])
    # print('-=-=-=-=-')
    # print('recieve_corr: ',recieve_corr)
recieve_corr = np.round(recieve_corr,1)
#ATL calc
ATL_val = srs_overalloct - recieve_corr+10*(np.log10(parition_area/sabines))
# print(ATL_val)


rec vs background: Unnamed: 14    27.62968
Unnamed: 15    30.56529
Unnamed: 16    24.04743
Unnamed: 17    19.50104
Unnamed: 18    26.08503
Unnamed: 19    28.03228
Unnamed: 20    27.56421
Unnamed: 21    21.41926
Unnamed: 22    23.10352
Unnamed: 23    20.64559
Unnamed: 24    19.44593
Unnamed: 25     16.3429
Unnamed: 26    16.46266
Unnamed: 27     20.8181
Unnamed: 28    21.48718
Unnamed: 29    23.46285
Name: 7, dtype: object
val: 27.629679999999993
count:  0
val: 30.565289999999997
count:  1
val: 24.04743
count:  2
val: 19.501040000000003
count:  3
val: 26.085030000000003
count:  4
val: 28.03228
count:  5
val: 27.564209999999996
count:  6
val: 21.41926
count:  7
val: 23.10352
count:  8
val: 20.64559
count:  9
val: 19.44593
count:  10
val: 16.3429
count:  11
val: 16.462660000000003
count:  12
val: 20.818100000000005
count:  13
val: 21.487179999999995
count:  14
val: 23.46285
count:  15


In [43]:
# print('constant',constant)
print(ATL_val)
ASTC_start = 20


Unnamed: 14    34.64041
Unnamed: 15    30.03485
Unnamed: 16    37.74359
Unnamed: 17    39.60853
Unnamed: 18    36.54972
Unnamed: 19    36.40224
Unnamed: 20     38.4996
Unnamed: 21     43.4475
Unnamed: 22    45.72243
Unnamed: 23    47.05425
Unnamed: 24    47.86501
Unnamed: 25    48.48977
Unnamed: 26    47.92896
Unnamed: 27    47.89255
Unnamed: 28    50.85702
Unnamed: 29    52.68486
Name: 7, dtype: object


In [66]:
# print(ASTC_start)
# print(ASTC_curve)
pos_diffs = list()
diff_negative=0
diff_positive=0 
ASTC_start = 20
New_curve =list()
new_sum = 0
STCCurve = [-16, -13, -10, -7, -4, -1, 0, 1, 2, 3, 4, 4, 4, 4, 4, 4]

while (diff_negative < 8 and new_sum < 32):
    # print('starting loop')
    print('ASTC fit test value: ', ASTC_start)
    for vals in STCCurve:
        New_curve.append(vals+ASTC_start)
    ASTC_curve = New_curve - ATL_val
    # print('ASTC curve: ',ASTC_curve)

    diff_negative =  np.max(ASTC_curve - ATL_val)
    print('Max, single diff: ', diff_negative)
    for val in ASTC_curve:
        if val > 0:
            pos_diffs.append(np.round(val))
        else:
            pos_diffs.append(0)
    # print(pos_diffs)
    new_sum = np.sum(pos_diffs)
    print('Sum Positive diffs: ', new_sum)
    
    if new_sum > 32 or diff_negative > 8:
        print('Curve too high! ASTC fit: ', ASTC_start-1) 
        break 
    pos_diffs = []
    New_curve = []
    ASTC_start = ASTC_start + 1
    
    
    if ASTC_start >80: break
    
# print(diff_negative)    
# print(ASTC_start)

ASTC fit test value:  20
Max, single diff:  -53.069699697817995
Sum Positive diffs:  0
ASTC fit test value:  21
Max, single diff:  -52.069699697817995
Sum Positive diffs:  0
ASTC fit test value:  22
Max, single diff:  -51.069699697817995
Sum Positive diffs:  0
ASTC fit test value:  23
Max, single diff:  -50.069699697817995
Sum Positive diffs:  0
ASTC fit test value:  24
Max, single diff:  -49.069699697817995
Sum Positive diffs:  0
ASTC fit test value:  25
Max, single diff:  -48.069699697817995
Sum Positive diffs:  0
ASTC fit test value:  26
Max, single diff:  -47.069699697817995
Sum Positive diffs:  0
ASTC fit test value:  27
Max, single diff:  -46.069699697817995
Sum Positive diffs:  0
ASTC fit test value:  28
Max, single diff:  -45.069699697817995
Sum Positive diffs:  0
ASTC fit test value:  29
Max, single diff:  -44.069699697817995
Sum Positive diffs:  0
ASTC fit test value:  30
Max, single diff:  -43.069699697817995
Sum Positive diffs:  0
ASTC fit test value:  31
Max, single diff: 

In [48]:
# print(STCCurve)    
print(ASTC_start)
New_curve =list()

# print(np.sum(pos_diffs))
print('starting loop')
STCCurve = [-16, -13, -10, -7, -4, -1, 0, 1, 2, 3, 4, 4, 4, 4, 4, 4]
for vals in STCCurve:
    New_curve.append(vals+ASTC_start)
print(New_curve)
print('ATL curve:', ATL_val)
ASTC_curve = New_curve - ATL_val
# print(ASTC_curve)
print('diff per freq:', ASTC_curve)
diff_negative =  np.max(ASTC_curve)
print('diff negative: ', diff_negative)
for diff in ASTC_curve:
    if diff > 0:
        pos_diffs.append(diff)
    else:
        pos_diffs.append(0)
new_sum = np.sum(pos_diffs)
print('positive diffs:',pos_diffs)
print('new sum: ',new_sum)
# print('diff negative:',diff_negative)
pos_diffs = []    
ASTC_start = ASTC_start + 1
print(ASTC_start)


21
starting loop
[5, 8, 11, 14, 17, 20, 21, 22, 23, 24, 25, 25, 25, 25, 25, 25]
ATL curve: Unnamed: 14    34.64041
Unnamed: 15    30.03485
Unnamed: 16    37.74359
Unnamed: 17    39.60853
Unnamed: 18    36.54972
Unnamed: 19    36.40224
Unnamed: 20     38.4996
Unnamed: 21     43.4475
Unnamed: 22    45.72243
Unnamed: 23    47.05425
Unnamed: 24    47.86501
Unnamed: 25    48.48977
Unnamed: 26    47.92896
Unnamed: 27    47.89255
Unnamed: 28    50.85702
Unnamed: 29    52.68486
Name: 7, dtype: object
diff per freq: Unnamed: 14   -29.64041
Unnamed: 15   -22.03485
Unnamed: 16   -26.74359
Unnamed: 17   -25.60853
Unnamed: 18   -19.54972
Unnamed: 19   -16.40224
Unnamed: 20    -17.4996
Unnamed: 21    -21.4475
Unnamed: 22   -22.72243
Unnamed: 23   -23.05425
Unnamed: 24   -22.86501
Unnamed: 25   -23.48977
Unnamed: 26   -22.92896
Unnamed: 27   -22.89255
Unnamed: 28   -25.85702
Unnamed: 29   -27.68486
Name: 7, dtype: object
diff negative:  -16.40223980554881
positive diffs: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [ ]:
# for many tests requested, iterate through
print(diff_negative)    
print(ASTC_start)
alltests = [
    '2.1',
    '2.2',
    '2.3'
]
# print(testplanfile['Test Label'])
for count, testnum in enumerate(alltests, start=1):
    print(count, testnum)


In [ ]:
### #### #### working v1 above - with template, still some work to process ASTC
Source_room_vol = 10080  # in cubic ft 
Recieve_room_vol = 12096 # in cubic ft
# write a function for getting the ASTC number from the data collected
df = pd.read_excel(rawReportpath+reports[1], sheet_name='SLM Data')
# STC_contour = [28,31,34,37,40,43,44,45,46,47,48,48,48,48,48,48]
# ATL = 


In [ ]:
# plotting all the L sites 

allrawpath = '//DLA-04/Shared/KAILUA PROJECTS/2022/22-058 P-209 EIS Noise Measurements/Noise Survey/Raw Data/'
site_list = ['L1: Lehua Elementary School',
             'L2: Pearl City Peninsula',
             'L3: Pier Side Child Development Center',
             'L4: Ford Island Child Development Center',
             'L5: Corner of Wasp and Chaffee on Ford Island',
             'L6: Corner of Crommelin and Ford Island Ave',
             'L7: Hickam Elementary School',
             'L8: JBPHH Housing',
             'L9: House of Worship']
slm_list = ['SLM-D_Longterm_L1/',
            'SLM-C_Longterm_L2/',
            'SLM-B_Longterm_L3/',
            'SLM-C_Longterm_L4 Redo/',
            'SLM-D_Longterm_L5 Redo/',
            'SLM-B_Longterm_L6 Redo/',
            'SLM-D_Longterm_L7/',
            'SLM-C_Longterm_L8/',
            'SLM-B_Longterm_L9/']
alldays = [['2023-02-22','2023-02-23'],
           ['2023-02-22','2023-02-23'],
           ['2023-02-22','2023-02-23'],
           ['2023-03-15','2023-03-16'],
           ['2023-03-15','2023-03-16'],
           ['2023-03-15','2023-03-16'],
           ['2023-03-21','2023-03-22'],
           ['2023-03-21','2023-03-22'],
           ['2023-03-21','2023-03-22']]

# join(allrawpath,slm_list[1])
for j in range(len(slm_list)):
    curr_archive = join(allrawpath,slm_list[j])
    archive_date = alldays[j]
    path_datafiles = [f for f in listdir(curr_archive) if isfile(join(curr_archive,f))]
    #only read excel files, filter for .xlsx
    path_datafiles = [k for k in path_datafiles if '.xlsx' in k]
    # read all xcel files into single dataframe
    datasheet = 'Measurement History'
    longterm = pd.DataFrame()
    for i in range(len(path_datafiles)):
        curr_longterm = pd.read_excel(curr_archive+path_datafiles[i],datasheet)
        longterm = pd.concat([longterm, curr_longterm])
    #remove periods from columns 
    longterm.columns = longterm.columns.str.replace(".","")
    longterm.columns = longterm.columns.str.replace("#","")
    longterm = longterm.set_index('Date')

    #####-=-=-=-=-plotting Line graphs ONLY -=-=--=-=#######
    # for days in archive_date:
    # currday = longterm.get(days)
    dayone = longterm.get(archive_date[0])
    dateone = archive_date[0] # string for date display
    daytwo =longterm.get(archive_date[1])
    datetwo = archive_date[1]
    # can we figure out how to loop through these? 

    # figure definitions - each figure has two plots, share the y scale 
    fig, [ax1, ax2] = plot.subplots(2,1, sharey=True, layout='constrained')
    fig.supxlabel("Date and Time of Measurement")
    fig.supylabel("Hourly Averaged Equivalent Sound Level, Leq (dBA)")

    # plot.plot(dayone.Time,dayone.LAeq, color = 'blue', linestyle = '-', label = 'LAeq')
    ax1.plot(dayone.Time,dayone.LAeq, color = 'blue', linestyle = '-', label = 'LAeq')
    # plot.plot(dayone.Time,dayone.LASmax, color = 'red', linestyle = ':', label = 'LASmax')
    # plot.plot(dayone.Time,dayone.LAS9000,color = 'green', linestyle = '--', label = 'LAS90')
  
    ax1.set_ylim(35, 80)
    ax1.set_yticks(np.arange(35,80,5))
    ax1.xaxis.set_major_locator(HourLocator(byhour=range(0,24,1)))
    ax1.xaxis.set_major_formatter(DateFormatter('%H'))
    ax1.grid(True)
    
    ax1.set_title(join(site_list[j],dateone)) 

    ax1.axvspan(dateone+' 00:00:00.000',dateone+' 07:00:00.000',
                color = 'grey', alpha = 0.5)
    ax1.axvspan(dateone+' 22:00:00.000',dateone+' 23:00:00.000',
                color = 'grey', alpha = 0.5)
    ax1.tick_params(axis='x', rotation = 55)
    
    #####-=--=-=-=- 2nd plot -=-=-=-==-######
    
    ax2.plot(daytwo.Time,daytwo.LAeq, color = 'blue', linestyle = '-', label = 'LAeq')
    # plot.plot(dayone.Time,dayone.LASmax, color = 'red', linestyle = ':', label = 'LASmax')
    # plot.plot(dayone.Time,dayone.LAS9000,color = 'green', linestyle = '--', label = 'LAS90')
 
    ax2.xaxis.set_major_locator(HourLocator(byhour=range(0,24,1)))
    ax2.xaxis.set_major_formatter(DateFormatter('%H'))
    ax2.grid(True)
    ax2.set_title(join(site_list[j],datetwo)) 

    ax2.axvspan(datetwo+' 00:00:00.000',datetwo+' 07:00:00.000',
                color = 'grey', alpha = 0.5)
    ax2.axvspan(datetwo+' 22:00:00.000',datetwo+' 23:00:00.000',
                color = 'grey', alpha = 0.5)
    ax2.tick_params(axis='x', rotation = 55)




    

In [ ]:

#plotting and statistics: 
for days in daystopull:
    dayone = longterm.get(days)
    dayone.columns = dayone.columns.str.replace(" #","")
    plot.figure(days)
    plot.plot(dayone.Time,dayone.LAeq, color = 'blue', linestyle = '-', label = 'LAeq')
    plot.plot(dayone.Time,dayone.LASmax, color = 'red', linestyle = ':', label = 'LASmax')
    plot.plot(dayone.Time,dayone.LAS9000,color = 'green', linestyle = '--', label = 'LAS90')
    plot.ylabel("Hourly Averaged Equivalent Sound Level, Leq (dBA)")
    plot.xlabel("Date and Time of Measurement")
    plot.grid(True)
    # plot a consistent placement for the legend?
    plot.legend()
    plot.axvspan(days+' 00:00:00.000',days+' 07:00:00.000',
                 color = 'grey', alpha = 0.5)
    plot.axvspan(days+' 22:00:00.000',days+' 23:00:00.000',
                 color = 'grey', alpha = 0.5)
    plot.tick_params(axis='x', rotation = 55)
    ## stats for day
    print('Day: ', days)
    print('LAeq min: ',min(dayone.LAeq))
    print('LAeq max: ',max(dayone.LAeq))
    daytime = dayone.between_time('07:00:00.000','21:00:00.000')
    nighttime = pd.concat([dayone.between_time('00:00:00.000','06:00:00.000'),dayone.between_time('22:00:00.000','23:00:00.000')])
    print('Daytime LAeq: ',np.mean(daytime.LAeq))
    daytime_mean = np.mean(daytime.LAeq)
    print('Nighttime LAeq: ',np.mean(nighttime.LAeq))
    night_mean = np.mean(nighttime.LAeq)
    Level_dn = 10*(np.log10((15*(10**(daytime_mean/10))+9*(10**((night_mean+10)/10)))/24))
    print('Ldn: ',Level_dn)
    print('-=-=-=-=-=-=-=-=-')

